**Capítulo 7: Ensamble Learning e Florestas Aleatórias**

O Capítulo 7 do livro "Hands-On Machine Learning with Scikit-Learn & TensorFlow" aborda o **Aprendizado em Conjunto (Ensemble Learning)** e as **Random Forests**, que são métodos poderosos para melhorar a precisão das previsões em Machine Learning. A ideia central é que, ao agregar as previsões de um grupo de preditores (como classificadores ou regressores), muitas vezes obtêm-se resultados superiores aos do melhor preditor individual, uma técnica conhecida como "sabedoria da multidão".

Os principais conceitos detalhados no capítulo são:

*   **Aprendizado em Conjunto (Ensemble Learning)**:
    *   Refere-se a técnicas que combinam as previsões de vários preditores para obter um desempenho melhor do que qualquer preditor individual.
    *   Frequentemente utilizado no final de um projeto de Machine Learning, após a construção de alguns bons preditores, para combiná-los em um preditor ainda melhor.
    *   É um fator comum nas soluções vencedoras de competições de Machine Learning.

*   **Classificadores por Votação (Voting Classifiers)**:
    *   Consiste em treinar vários classificadores diferentes (por exemplo, Regressão Logística, SVM, Random Forest) e agregar suas previsões.
    *   A eficácia advém do fato de que, se os classificadores forem suficientemente independentes e cometerem erros não correlacionados, a precisão do conjunto pode ser significativamente maior do que a de cada classificador individual.
    *   Existem dois tipos principais:
        *   **Votação Dura (Hard Voting)**: O classificador simplesmente conta os votos de cada preditor e escolhe a classe que recebe a maioria dos votos.
        *   **Votação Suave (Soft Voting)**: Calcula a média das probabilidades de classe estimadas para cada classe e escolhe a classe com a probabilidade mais alta. Dá mais peso aos votos de alta confiança e geralmente tem melhor desempenho, mas requer que cada classificador seja capaz de estimar probabilidades de classe.

*   **Bagging e Pasting**:
    *   São métodos para obter um conjunto diverso de classificadores usando o **mesmo algoritmo de treinamento** para cada preditor, mas treinando-os em diferentes subconjuntos aleatórios do conjunto de treinamento.
    *   **Bagging (Bootstrap Aggregating)**: A amostragem dos subconjuntos é feita **com substituição**. Isso significa que a mesma instância de treinamento pode ser amostrada várias vezes no mesmo subconjunto.
    *   **Pasting**: A amostragem é feita **sem substituição**.
    *   Processo: Os preditores podem ser treinados em paralelo, o que os torna escaláveis.
    *   Agregação: Para classificação, a função de agregação é tipicamente a **moda** (previsão mais frequente); para regressão, é a **média**.
    *   Impacto: Cada preditor individualmente tem um viés maior, mas a agregação **reduz tanto o viés quanto a variância** do conjunto, resultando em um viés semelhante e uma variância menor em comparação com um único preditor treinado no conjunto de treinamento original.
    *   Bagging geralmente é preferido por introduzir mais diversidade e produzir modelos ligeiramente melhores, mesmo com um viés ligeiramente maior.

*   **Random Forests**:
    *   É um tipo de conjunto (ensemble) baseado em Bagging, mas que adiciona **aleatoriedade extra** além da amostragem de instâncias.
    *   Cada **Árvore de Decisão** (o preditor base) em uma Random Forest é treinada não apenas em um subconjunto aleatório de instâncias, mas também considerando um **subconjunto aleatório de atributos** em cada divisão do nó. Isso cria árvores ainda mais diversas e reduz ainda mais a variância do conjunto.
    *   É um dos algoritmos de Machine Learning mais poderosos disponíveis atualmente.
    *   **Importância de Atributos (Feature Importance)**: Random Forests podem indicar a importância relativa de cada atributo para fazer previsões precisas, sendo útil para seleção de atributos.

*   **Boosting**:
    *   A ideia principal é treinar preditores **sequencialmente**, onde cada novo preditor tenta **corrigir os erros** de seus antecessores.
    *   Diferente do Bagging/Pasting, o treinamento não pode ser totalmente paralelizado devido à natureza sequencial.
    *   **AdaBoost (Adaptive Boosting)**:
        *   Um classificador base (e.g., uma Árvore de Decisão) é treinado, e as instâncias de treinamento mal classificadas têm seus **pesos relativos aumentados**.
        *   Os classificadores subsequentes focam mais nos casos "difíceis".
        *   As previsões finais são feitas por uma **votação ponderada**, onde preditores mais precisos têm maior peso.
    *   **Gradient Boosting (Gradient Boosted Regression Trees - GBRT)**:
        *   Cada preditor (geralmente uma Árvore de Decisão) é treinado nos **erros residuais** (diferenças entre as previsões do conjunto atual e os valores reais) do preditor anterior.
        *   O hiperparâmetro `learning_rate` (taxa de aprendizado) escala a contribuição de cada árvore. Um valor baixo (shrinkage) requer mais árvores, mas melhora a generalização, atuando como uma técnica de regularização.
        *   A **parada antecipada (early stopping)** é comumente usada para encontrar o número ideal de árvores e evitar o sobreajuste.

*   **Stacking (Stacked Generalization)**:
    *   Em vez de usar funções triviais (como votação) para agregar previsões, o Stacking **treina um modelo** para realizar essa agregação.
    *   Esse modelo de agregação é chamado de **blender** ou **meta-learner**.
    *   O treinamento do blender geralmente envolve um **conjunto de validação separado (hold-out set)**:
        *   Os preditores da primeira camada são treinados em um subconjunto do conjunto de treinamento.
        *   Esses preditores então fazem previsões sobre o conjunto de hold-out.
        *   Essas previsões se tornam as *features de entrada* para o blender, que é treinado para fazer a previsão final.
    *   Alternativamente, pode-se usar **previsões out-of-fold**.

### **Implementação**

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesClassifier
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris

In [2]:
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC(probability=True)

In [3]:
X, y = make_moons(n_samples=1000, noise=0.3, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

voting_clf = VotingClassifier(estimators=[
    ('lr', log_clf),
    ('rf', rnd_clf),
    ('svc', svm_clf)
], voting='soft')
voting_clf.fit(X_train, y_train)


VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()),
                             ('svc', SVC(probability=True))],
                 voting='soft')

Diferença entre Soft Voting e Hard Voting

- **Hard Voting**: Cada classificador faz uma previsão (classe) e o voto da maioria decide o resultado final. Não considera as probabilidades, apenas as classes previstas.
- **Soft Voting**: Cada classificador retorna as probabilidades para cada classe. As probabilidades são somadas e a classe com maior probabilidade total é escolhida. Soft voting geralmente resulta em melhor desempenho quando os classificadores podem estimar probabilidades confiáveis.

In [4]:
for clf in (log_clf, rnd_clf, svm_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

y_pred_ensemble = voting_clf.predict(X_test)
print("VotingClassifier", accuracy_score(y_test, y_pred_ensemble))

LogisticRegression 0.82
RandomForestClassifier 0.9
SVC 0.915
VotingClassifier 0.92


**Bagging e Pasting no Scikit-Learn**

- **Bagging (Bootstrap Aggregating)**: Técnica de ensemble em que vários classificadores são treinados em subconjuntos aleatórios (com reposição) do conjunto de dados original. O objetivo é reduzir o overfitting e aumentar a robustez do modelo. O resultado final é obtido pela média (regressão) ou votação (classificação) das previsões dos modelos.

- **Pasting**: Semelhante ao bagging, mas os subconjuntos de dados são amostrados **sem reposição**. Ou seja, cada instância do conjunto de dados pode aparecer em apenas um subconjunto.

Ambas as técnicas ajudam a reduzir a variância dos modelos e são especialmente eficazes com algoritmos de alta variância, como árvores de decisão.

In [5]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1, oob_score=True, random_state=42)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)
print("Bagging accuracy:", accuracy_score(y_test, y_pred))

Bagging accuracy: 0.91


O que é oob_score?

- **oob_score** (out-of-bag score) é uma métrica de validação interna utilizada em métodos de ensemble baseados em bagging, como o Random Forest e o BaggingClassifier.
- Durante o treinamento, cada estimador é treinado em uma amostra aleatória do conjunto de dados (com reposição). As instâncias que não são selecionadas para o treinamento de um estimador específico são chamadas de "out-of-bag" (fora da amostra).
- O modelo pode ser avaliado nessas instâncias oob, funcionando como uma validação cruzada interna, sem a necessidade de um conjunto de validação separado.
- Para usar, basta definir `oob_score=True` ao criar o classificador. Após o treinamento, o atributo `oob_score_` fornecerá a acurácia estimada usando as amostras oob.

In [6]:
bag_clf.oob_decision_function_

array([[0.03496503, 0.96503497],
       [0.96270396, 0.03729604],
       [0.98868778, 0.01131222],
       ...,
       [0.97031963, 0.02968037],
       [0.72566372, 0.27433628],
       [0.05643341, 0.94356659]], shape=(800, 2))

***Florestas Aleatórias***

In [7]:
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=42)
rnd_clf.fit(X_train, y_train)

y_pred = rnd_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Random Forest acurácia:", accuracy)

Random Forest acurácia: 0.925


In [8]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(splitter="random", max_leaf_nodes=16), n_estimators=500,
    max_samples=1.0, bootstrap=True, n_jobs=-1, oob_score=True, random_state=42)

**Importância das características**

In [9]:
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=42)

In [10]:
rnd_clf.fit(iris["data"], iris["target"])

RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=42)

In [11]:
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(f"Feature: {name}, Score: {score}")


Feature: sepal length (cm), Score: 0.11249225099876375
Feature: sepal width (cm), Score: 0.02311928828251033
Feature: petal length (cm), Score: 0.4410304643639577
Feature: petal width (cm), Score: 0.4233579963547682


**Boosting**

In [12]:
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200, algorithm="SAMME", learning_rate=0.5, random_state=42)
ada_clf.fit(X_train, y_train)
y_pred = ada_clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

c:\Users\andre\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(


Accuracy: 0.925


In [13]:
tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X, y)

DecisionTreeRegressor(max_depth=2)

In [14]:
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2)

In [15]:
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2)

In [16]:
X_new = X[:5]  

y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))
print("Predictions:", y_pred)

Predictions: [0.22156058 0.95678584 0.85922625 1.09570306 0.06013363]


In [16]:
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X, y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

In [17]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=49)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred) for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors)

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=np.int64(97))

In [18]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range (1,120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
    if error_going_up == 5:
        break

### **Exercícios**

1) Se você treinou cinco modelos diferentes com exatamente os mesmos dados de treinamento e todos conseguem uma precisão de 95%, existe alguma chance de você combinar esses modelos para obter melhores resultados? Em caso afirmativo, como? Se não, por quê?

Sim, existe a possibilidade de combinar esses cinco modelos para obter melhores resultados, mesmo que todos tenham sido treinados com os mesmos dados e apresentem a mesma precisão individual. Isso é possível se os modelos cometerem erros diferentes (ou seja, se seus erros não forem perfeitamente correlacionados). Ao utilizar técnicas de ensemble, como o **Voting Classifier** (classificação por votação) ou métodos como **Bagging** e **Stacking**, é possível que o conjunto dos modelos corrija os erros individuais, resultando em uma performance geral melhor do que qualquer modelo isolado. Essa melhoria ocorre devido ao efeito da "sabedoria da multidão", onde a combinação de opiniões (ou previsões) tende a ser mais robusta e precisa, especialmente se os modelos forem suficientemente diversos.

2. Qual a diferença entre os classificadores de votação hard e soft?

A principal diferença entre os classificadores de votação hard e soft está na forma como as previsões dos modelos são combinadas:

- **Hard Voting:** Cada classificador faz uma previsão de classe e o resultado final é decidido pela maioria dos votos (classe mais votada). Não leva em conta a confiança (probabilidade) de cada predição, apenas a classe prevista por cada modelo.

- **Soft Voting:** Cada classificador retorna as probabilidades para cada classe. As probabilidades são somadas e a classe com maior probabilidade total é escolhida como resultado final. Soft voting geralmente apresenta melhor desempenho, pois considera o grau de confiança de cada modelo nas suas previsões, desde que os classificadores consigam estimar probabilidades confiáveis.

3. É possível acelerar o treinamento de um bagging ensamble distribuindo-o por vários servidores? E com pasting ensambles, boosting ensembles, florestas aleatórias, ou stacking ensembles?

Sim, é possível acelerar o treinamento de alguns tipos de ensembles distribuindo-os por vários servidores, dependendo do método utilizado:

- **Bagging Ensembles (e.g., BaggingClassifier, Random Forests):**  
    O treinamento pode ser facilmente distribuído, pois cada modelo é treinado de forma independente em subconjuntos diferentes dos dados. Isso permite que os modelos sejam treinados em paralelo em diferentes servidores.

- **Pasting Ensembles:**  
    Assim como o Bagging, o treinamento pode ser distribuído, já que os modelos também são treinados de forma independente em subconjuntos diferentes dos dados, sem reposição.

- **Boosting Ensembles (e.g., AdaBoost, Gradient Boosting):**  
    O treinamento não pode ser totalmente distribuído, pois os modelos são treinados sequencialmente. Cada modelo depende dos erros do modelo anterior, o que impede a paralelização completa.

- **Random Forests:**  
    Como é uma técnica baseada em Bagging, o treinamento das árvores pode ser distribuído por vários servidores, já que cada árvore é treinada de forma independente.

- **Stacking Ensembles:**  
    O treinamento dos modelos da primeira camada pode ser distribuído, pois eles são treinados de forma independente. No entanto, o treinamento do meta-modelo (blender) não pode ser distribuído, pois depende das previsões dos modelos da primeira camada.

Em resumo, métodos baseados em Bagging (como Random Forests) e Pasting são mais adequados para paralelização e distribuição, enquanto Boosting e Stacking têm limitações devido à sua natureza sequencial ou dependência entre camadas.

4. Qual é o benefício da avaliação out-of-bag?

**Benefício da avaliação out-of-bag (oob_score):**

A avaliação out-of-bag (oob_score) oferece uma maneira eficiente de validar o desempenho de modelos baseados em bagging, como Random Forests e BaggingClassifier, sem a necessidade de um conjunto de validação separado. Durante o treinamento, cada estimador é treinado em uma amostra aleatória do conjunto de dados (com reposição), deixando algumas instâncias fora da amostra (out-of-bag). Essas instâncias são usadas para avaliar o modelo, fornecendo uma estimativa de desempenho que é:

- **Eficiente:** Não requer dividir os dados em conjuntos de treinamento e validação, permitindo o uso de todos os dados para treinamento.
- **Confiável:** Fornece uma estimativa de desempenho semelhante à validação cruzada.
- **Automática:** A avaliação é realizada internamente durante o treinamento, sem necessidade de configuração adicional.

Isso ajuda a economizar tempo e maximizar o uso dos dados disponíveis.

5. O que torna as Árvores-Extras mais aleatórias do que as Florestas Aleatórias comuns? Como esta aleatoriedade extra pode ajudar? As Árvores-Extras são mais lentas ou mais rápidas do que as florestas Aleatórias regulares?

As Árvores-Extras (ExtraTrees) são mais aleatórias do que as Florestas Aleatórias comuns porque, além de selecionar aleatoriamente o subconjunto de atributos em cada divisão (como as Random Forests), elas também escolhem aleatoriamente o ponto de divisão para cada atributo, em vez de buscar o melhor ponto de corte. Essa aleatoriedade extra faz com que as árvores individuais sejam ainda mais diferentes entre si.

**Como essa aleatoriedade extra pode ajudar?**
- Ela reduz ainda mais a correlação entre as árvores do ensemble, o que pode diminuir a variância do modelo e ajudar a evitar overfitting, especialmente em conjuntos de dados ruidosos.

**Desempenho:**
- As Árvores-Extras geralmente são **mais rápidas** para treinar do que as Florestas Aleatórias comuns, pois não procuram o melhor ponto de divisão, apenas escolhem um ponto aleatório. Isso reduz o custo computacional de cada divisão.
- Em termos de precisão, podem ser ligeiramente menos precisas em alguns casos, mas a diferença costuma ser pequena e, em muitos cenários, o ganho de velocidade compensa.

6. Se o seu ensemble AdaBoost se subajusta aos dados de treinamento, quais hiperparâmetros você deve ajustar e como?

Se o ensemble AdaBoost estiver subajustando os dados de treinamento, você pode ajustar os seguintes hiperparâmetros:

- **`n_estimators` (número de estimadores):**  
    Aumente o número de estimadores para permitir que o modelo aprenda mais padrões nos dados. Um número maior de estimadores pode ajudar a reduzir o subajuste.

- **`learning_rate` (taxa de aprendizado):**  
    Aumente a taxa de aprendizado para que cada estimador contribua mais para o modelo final. Isso pode ajudar o modelo a se ajustar melhor aos dados.

- **`base_estimator` (estimador base):**  
    Use um modelo base mais complexo, como uma árvore de decisão com maior profundidade (`max_depth`). Por padrão, o AdaBoost usa árvores de decisão rasas (stumps), que podem ser insuficientes para dados complexos.

- **`algorithm`:**  
    Certifique-se de que o algoritmo está configurado corretamente. Por exemplo, o `SAMME.R` (padrão) geralmente funciona melhor para problemas de classificação.

7. Se o seu ensemble AdaBoost se sobreajusta ao conjunto de treinamento, você deve ajustar como?

Se o ensemble AdaBoost estiver se sobreajustando ao conjunto de treinamento, você pode ajustar os seguintes hiperparâmetros para reduzir o overfitting:

- **`n_estimators` (número de estimadores):**  
    Reduza o número de estimadores para evitar que o modelo fique excessivamente complexo.

- **`learning_rate` (taxa de aprendizado):**  
    Diminua a taxa de aprendizado para que cada estimador contribua menos para o modelo final, o que pode ajudar a melhorar a generalização.

- **`base_estimator` (estimador base):**  
    Use um modelo base mais simples, como uma árvore de decisão mais rasa (`max_depth=1`), para reduzir a complexidade do modelo.

- **`algorithm`:**  
    Certifique-se de que o algoritmo está configurado corretamente. O `SAMME.R` (padrão) geralmente é mais robusto e pode ajudar a evitar overfitting em comparação com o `SAMME`.

8. Carregue os dados MNIST (introduzido no Capítulo 3) e o divida em um conjunto de treinamento, um conjunto de validação e um conjunto de teste (por exemplo, utilize 40 mil instâncias para treinamento, 10 mil para validação e 10 mil para teste). Em seguida, treine vários classificadores como um classificador Floresta Aleatória, um classificador Árvores-extra e um SVM. Em seguida, utilizando um classificador de votação soft ou hard, tente combina-los em um ensamble, teste-o no conjunto de teste. Qual é a melhoria de desempenho em comparação com os classificadores individuais?

Tempo computacional muito grande:

In [ ]:
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist["data"], mnist["target"]
y = y.astype(int)

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=10000, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=10000, random_state=42)

# Random Forest
rnd_clf = RandomForestClassifier(n_estimators=10000, random_state=42)
rnd_clf.fit(X_train, y_train)

# Extra Trees
ext_clf = ExtraTreesClassifier(n_estimators=10000, random_state=42)
ext_clf.fit(X_train, y_train)

# SVM
svm_clf = SVC(probability=True, random_state=42) #probability = false, funciona quando o "voting" do ensemble estiver configurado no voting = "hard".
svm_clf.fit(X_train, y_train)

voting_clf = VotingClassifier(
    estimators=[('rf', rnd_clf), ('et', ext_clf), ('svc', svm_clf)],
    voting='soft'
)
voting_clf.fit(X_train, y_train)

for clf in (rnd_clf, ext_clf, svm_clf, voting_clf):
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, "accuracy:", accuracy_score(y_test, y_pred))

Algoritmo otimizado para ser mais rápido:

In [ ]:
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist["data"], mnist["target"].astype(int)

X_small, _, y_small, _ = train_test_split(X, y, train_size=3000, stratify=y, random_state=42)
X_train, X_temp, y_train, y_temp = train_test_split(X_small, y_small, train_size=2000, stratify=y_small, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, train_size=250, stratify=y_temp, random_state=42)

# Random Forest
rnd_clf = RandomForestClassifier(n_estimators=50, random_state=42)
rnd_clf.fit(X_train, y_train)

# Extra Trees
ext_clf = ExtraTreesClassifier(n_estimators=50, random_state=42)
ext_clf.fit(X_train, y_train)

# SVM (rápido com poucos dados)
svm_clf = SVC(probability=False, random_state=42)
svm_clf.fit(X_train, y_train)

# Ensemble
voting_clf = VotingClassifier(
    estimators=[('rf', rnd_clf), ('et', ext_clf), ('svc', svm_clf)],
    voting='hard'
)
voting_clf.fit(X_train, y_train)

for clf in (rnd_clf, ext_clf, svm_clf, voting_clf):
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, "accuracy:", accuracy_score(y_test, y_pred))

RandomForestClassifier accuracy: 0.904
ExtraTreesClassifier accuracy: 0.9186666666666666
SVC accuracy: 0.9373333333333334
VotingClassifier accuracy: 0.9306666666666666


9. Execute os classificadores individuais do exercício anterior para fazer previsões no conjunto de validação e crie um novo conjunto de treinamento com as previsões resultantes: Cada instância de treinamento é um vetor que contém o conjunto de previsões de todos os seus classificadores para uma imagem e o alvo é a classe da imagem. Parabéns, você acabou de treinar um blender, e, junto com os classificadores, eles formam um stacking ensemble! A gora, avaliaremos o conjunto no conjunto de testes. Para cada imagem no conjunto de teste, faça previsões com todos os classificadores, então forneça as previsões ao blender para obter as previsões do ensemble. Como ela se compara ao classificador de votação que você treinou anteriormente?

In [20]:
val_pred_rf = rnd_clf.predict(X_val)
val_pred_et = ext_clf.predict(X_val)
val_pred_svm = svm_clf.predict(X_val)

X_blender_train = np.c_[(val_pred_rf, val_pred_et, val_pred_svm)]
y_blender_train = y_val

blender = LogisticRegression(max_iter=1000, random_state=42)
blender.fit(X_blender_train, y_blender_train)

test_pred_rf = rnd_clf.predict(X_test)
test_pred_et = ext_clf.predict(X_test)
test_pred_svm = svm_clf.predict(X_test)

X_blender_test = np.c_[(test_pred_rf, test_pred_et, test_pred_svm)]

y_pred_blender = blender.predict(X_blender_test)
blender_acc = accuracy_score(y_test, y_pred_blender)
print('Stacking Ensemble (Blender) accuracy:', blender_acc)

Stacking Ensemble (Blender) accuracy: 0.8253333333333334
